In [4]:
!pip install scipy
from scipy.stats import jarque_bera
import matplotlib.dates as mdates

# =============================================================================
# 2.1 GRÁFICOS DE PRECIOS NORMALIZADOS
# =============================================================================

print("\n\n2.1 GRÁFICOS DE PRECIOS NORMALIZADOS")
print("-" * 60)

precios_norm = precios_cierre / precios_cierre.iloc[0] * 100

plt.figure(figsize=(14, 7))
for ticker in tickers_cartera:
    plt.plot(precios_norm.index, precios_norm[ticker], label=ticker, linewidth=2)

plt.title('Evolución de Precios Normalizados - Base 100', fontsize=14)
plt.xlabel('Fecha')
plt.ylabel('Precio Normalizado (Base 100)')
plt.legend()
plt.grid(True)
plt.show()

# =============================================================================
# 2.2 EVENTOS QUE IMPACTARON SIGNIFICATIVAMENTE EN PRECIOS
# =============================================================================

print("\n\n2.2 EVENTOS MACROECONÓMICOS/CORPORATIVOS DESTACADOS")
print("-" * 60)

print("""
✓ Marzo 2020: COVID-19 y desplome del precio del petróleo ➜ Caídas abruptas en todo el sector energético.
✓ Octubre 2023: Elecciones presidenciales en Argentina ➜ Alta volatilidad en ADRs argentinos como YPF y PAM.
✓ Febrero 2022: Inicio del conflicto Rusia-Ucrania ➜ Incremento del precio del crudo ➜ Impacto positivo en petroleras como PBR y VIST.
""")

# Opcional: marcar eventos en el gráfico con líneas verticales
eventos = {
    '2020-03-01': 'COVID-19',
    '2022-02-24': 'Rusia-Ucrania',
    '2023-10-22': 'Elecciones ARG'
}

plt.figure(figsize=(14, 7))
for ticker in tickers_cartera:
    plt.plot(precios_norm.index, precios_norm[ticker], label=ticker)

for fecha, evento in eventos.items():
    plt.axvline(pd.to_datetime(fecha), color='red', linestyle='--', alpha=0.6)
    plt.text(pd.to_datetime(fecha), 150, evento, rotation=90, fontsize=9)

plt.title('Eventos Macroeconómicos sobre Precios Normalizados', fontsize=14)
plt.legend()
plt.grid(True)
plt.show()

# =============================================================================
# 2.3 ANÁLISIS DE ESTACIONALIDAD POR TRIMESTRES
# =============================================================================

print("\n\n2.3 ANÁLISIS DE ESTACIONALIDAD POR TRIMESTRES")
print("-" * 60)

# Añadir columna de trimestre
retornos_diarios['Trimestre'] = retornos_diarios.index.to_period("Q")

# Preparar datos para boxplot
retornos_temp = retornos_diarios.drop(columns='Trimestre')

# Melt del DataFrame
retornos_diarios_melt = retornos_temp.reset_index().melt(id_vars='Date', var_name='Ticker', value_name='Retorno')

# Agregar trimestre a partir de la columna de fecha
retornos_diarios_melt['Trimestre'] = pd.to_datetime(retornos_diarios_melt['Date']).dt.to_period('Q')

# Gráfico de boxplot
plt.figure(figsize=(12, 6))
sns.boxplot(data=retornos_diarios_melt, x='Trimestre', y='Retorno', hue='Ticker')
plt.title('Distribución de Retornos por Trimestre')
plt.xlabel('Trimestre')
plt.ylabel('Retorno Diario (%)')
plt.legend(loc='upper right')
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

# =============================================================================
# 2.4 MÉTRICAS DE RIESGO
# =============================================================================

print("\n\n2.4 CÁLCULO DE MÉTRICAS DE RIESGO")
print("-" * 60)

riesgo_df = pd.DataFrame(index=tickers_cartera)

# VaR 95% (histórico)
riesgo_df['VaR 95%'] = retornos_diarios[tickers_cartera].quantile(0.05)

# CVaR (Expected Shortfall)
riesgo_df['CVaR 95%'] = retornos_diarios[tickers_cartera].apply(lambda x: x[x <= x.quantile(0.05)].mean())

# Drawdown máximo
max_drawdown = {}
for ticker in tickers_cartera:
    serie = precios_cierre[ticker]
    rolling_max = serie.cummax()
    drawdown = (serie - rolling_max) / rolling_max
    max_drawdown[ticker] = drawdown.min()

riesgo_df['Max Drawdown'] = pd.Series(max_drawdown)

# Mostrar tabla
print(riesgo_df.round(4))

# =============================================================================
# 2.5 TEST DE NORMALIDAD Y ANÁLISIS DE COLAS
# =============================================================================

print("\n\n2.5 TEST DE NORMALIDAD Y COLAS (Jarque-Bera)")
print("-" * 60)

for ticker in tickers_cartera:
    jb_stat, jb_p = jarque_bera(retornos_diarios[ticker])
    print(f"{ticker}: Jarque-Bera = {jb_stat:.2f}, p-valor = {jb_p:.4f} {'(No Normal)' if jb_p < 0.05 else '(Normal)'}")

# Distribuciones
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 8))
axes = axes.flatten()

for i, ticker in enumerate(tickers_cartera):
    sns.histplot(retornos_diarios[ticker], kde=True, bins=100, ax=axes[i])
    axes[i].set_title(f"Distribución de Retornos - {ticker}")
    axes[i].grid(True)

plt.tight_layout()
plt.show()

   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
   - -------------------------------------- 1.3/38.5 MB 6.7 MB/s eta 0:00:06
   -- ------------------------------------- 2.9/38.5 MB 7.0 MB/s eta 0:00:06
   ---- ----------------------------------- 4.5/38.5 MB 7.5 MB/s eta 0:00:05
   ------ --------------------------------- 6.0/38.5 MB 7.2 MB/s eta 0:00:05
   ------- -------------------------------- 7.6/38.5 MB 7.2 MB/s eta 0:00:05
   -------- ------------------------------- 8.7/38.5 MB 6.9 MB/s eta 0:00:05
   --------- ------------------------------ 9.4/38.5 MB 6.6 MB/s eta 0:00:05
   ---------- ----------------------------- 10.0/38.5 MB 6.1 MB/s eta 0:00:05
   ---------- ----------------------------- 10.5/38.5 MB 5.8 MB/s eta 0:00:05
   ----------- ---------------------------- 10.7/38.5 MB 5.5 MB/s eta 0:00:06
   ----------- ---------------------------- 11.3/38.5 MB 4.9 MB/s eta 0:00:06
   ----------- ---------------------------- 11.3/38.5 MB 4.9 MB/s eta 0:00:06
 


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip




2.1 GRÁFICOS DE PRECIOS NORMALIZADOS
------------------------------------------------------------


NameError: name 'precios_cierre' is not defined